In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.ndimage.filters import gaussian_filter
import pandas as pd
import scipy.ndimage as sp
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from cycler import cycler
import scipy.stats as stats

from skimage import io, exposure, data
from skimage import feature

mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['pdf.fonttype'] = 42

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



#----------MINFLUX tracks NP array to Pandas-----------------

pd.set_option('display.float_format', '{:.20f}'.format)


#import numpy raw data

directory = '...'
minflux = np.load('directory.npy')
minf = pd.DataFrame({'track_id': minflux['tid'], 'time' : minflux['tim']})

pos = pd.DataFrame((minflux['itr']['loc'][:,3]), columns = ['X','Y','Z'])


min_data = pd.concat([minf, pos], axis=1)
min_data_s = min_data.sort_values(by=['track_id', 'time'])



#-----------coordinate conversion- Lengths and offsets---------------
#Length and offset are copied from metadata


L = [1.4399999599845614e-05, 1.739999970595818e-05, 1.0, 1.0]
O = [-2.5516666482872097e-05, -2.6513332159083802e-05, -0.5, -0.5]



Lengths = [i * 1e6 for i in L]
Offsets = [i * 1e6 for i in O]
min_data_s.X = (min_data_s.X*1e6)-Offsets[0]
min_data_s.Y = (min_data_s.Y*1e6)-Offsets[1]


param = pd.DataFrame({'L': [L], 'O':[O], 'filename':min_filename})
param = param.T
#save track data in csv format
min_data_s.to_csv('.../spots.csv')
#save parameter file in csv format
param.to_csv('.../params.csv')


#--------import LD image----------------
#pre and post tracking confocal images are exported and combined in a maximum intensity projection

cmap3 = LinearSegmentedColormap.from_list('mycmap', ['white', 'darkgreen'])
min_LDs = '/Volumes/PRO-G40/Arda/Arda/' + min_filename + '/LDs.tif'

IMAGE = io.imread(min_LDs)
pre = IMAGE[0]
post = IMAGE[1]
MIP= np.max(IMAGE, axis=0)
image_minmax_scaled = exposure.rescale_intensity(MIP)
percentiles = np.percentile(image_minmax_scaled, (70, 99.8))
scaled = exposure.rescale_intensity(MIP,
                                    in_range=tuple(percentiles))
pre_LDs = exposure.rescale_intensity(pre,
                                    in_range=tuple(percentiles))
post_LDs = exposure.rescale_intensity(post,
                                    in_range=tuple(percentiles))



plt.imshow(scaled, cmap=cmap3, extent=[0,Lengths[0],Lengths[1],0])
plt.xlabel('X (µm)')
plt.ylabel('Y (µm)')
plt.show()

In [ ]:
#display all tracks

plt.imshow(scaled, cmap=cmap3, extent=[0,Lengths[0],Lengths[1],0])
sns.scatterplot(data= min_data_s, x = min_data_s.X, y = min_data_s.Y, hue = 'track_id', s = 0.01, alpha = 0.85,  
                legend = False, palette = "Set2", edgecolor=None)
sns.lineplot(data= min_data_s, x = min_data_s.X, y = min_data_s.Y, lw = 0.05, sort = False, estimator = None, 
             hue = 'track_id', palette = 'Set2', legend = False, alpha = 0.6)

plt.xlim(0,Lengths[0])
plt.ylim(0, Lengths[1])

plt.xlabel ('X position (µm)')
plt.ylabel ('Y position (µm)')
#plt.axis('scaled')
plt.gca().invert_yaxis()

In [ ]:
#-----masking for lipid droplets and classifying tracks based on subcellular location-------

import cv2
from skimage import measure

threshold_value = 150

coor = pd.DataFrame({'track_id': min_data_s.track_id, 'X': min_data_s.X*20, 'Y': min_data_s.Y*20, 'time':min_data_s.time}, columns=['track_id','X', 'Y', 'time'])      

def isolate_puncta(input_image_path, output_mask_path, output_expanded_path, output_coordinates_path, threshold_value):
    # Load the image
    image = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply a threshold to separate puncta from the background
    _, binary_mask = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)

    # Label connected components in the binary mask
    labeled_mask, num_labels = measure.label(binary_mask, connectivity=2, return_num=True)

    # Get the properties of each labeled region (e.g., area, centroid)
    region_properties = measure.regionprops(labeled_mask)

    # Create a blank image to draw isolated puncta 
    isolated_puncta = np.zeros_like(image)

    # Collect pixel coordinates of isolated puncta
    coordinates = []

    # Iterate through labeled regions and isolate puncta
    for region in region_properties:
        if region.area > 5:  # Adjust the area threshold as needed
            # Draw the isolated punctum on the blank image
            isolated_puncta[labeled_mask == region.label] = 255

            # Collect pixel coordinates of the isolated puncta
            coordinates.extend(region.coords.tolist())

    # Expand the isolated mask by 3 pixels
    kernel = np.ones((3, 3), np.uint8)
    expanded_mask = cv2.dilate(isolated_puncta, kernel, iterations=1)
    
    
    # Save the binary mask, isolated puncta, and pixel coordinates
    cv2.imwrite(output_mask_path, binary_mask)
    cv2.imwrite(output_isolated_path, isolated_puncta)
    cv2.imwrite(output_expanded_path, expanded_mask)
    
    # Create a DataFrame from the pixel coordinates
    df_coordinates = pd.DataFrame(coordinates, columns=["Y", "X"])
    
    # Save the pixel coordinates as a CSV file
    df_coordinates.to_csv(output_coordinates_path, index=False)

if __name__ == "__main__":
    # Set your input and output paths
    directory_mask = '...'
    input_image_path = directory_mask + "MAX_LDs.tif"
    output_mask_path = directory_mask +"mask.png"
    output_isolated_path = directory_mask +"isolated_puncta.png"
    output_expanded_path = directory_mask +"expanded_puncta.png"
    output_coordinates_path = directory_mask +"coordinates.csv"

    # Set the threshold value (you may need to adjust this)
    threshold_value = threshold_value
    
    # Call the function to isolate puncta and collect coordinates
    isolate_puncta(input_image_path, output_mask_path, output_isolated_path, output_coordinates_path, threshold_value)
    mask_coordinates = pd.read_csv(output_coordinates_path, low_memory=False)
    
    inside = pd.DataFrame()
    for key,grp in mask_coordinates.groupby('X'):
        coord = coor[((coor['Y']//1).isin(grp.Y)) & ((coor['X']//1)==key) ]
        inside = pd.concat([inside, coord])
    
        
    outside = pd.concat([coor,inside]).drop_duplicates(keep=False)
    
    inside.X = inside.X/20
    inside.Y = inside.Y/20
    outside.X = outside.X/20
    outside.Y = outside.Y/20
    cmap_k = LinearSegmentedColormap.from_list('mycmap', ['white', 'black'])


    
inside['Loc'] = 'inside'
outside['Loc'] = 'outside'

MIN_track_info = pd.concat([outside, inside], ignore_index=True)

plt.imshow(scaled, cmap=cmap3, extent=[0,Lengths[0],Lengths[1],0])
sns.scatterplot(data= MIN_track_info, x = 'X', y = 'Y', hue = 'Loc', s = 0.05, alpha = 0.7, 
                 legend = True, palette = 'magma',  linewidth=0)
plt.title('Threshold value: ' + str(threshold_value))


plt.xlabel('X (µm)')
plt.xlabel('Y (µm)')
plt.xlim(0,Lengths[0])
plt.ylim(Lengths[1], 0)


plt.show()


In [ ]:
#----------MSD analysis for rach single molecule track--------------

msd_info = pd.DataFrame()


for track_id, group in MIN_track_info.groupby(['filename','track_id', 'Loc']):
    
    n = len(group)
    msd = np.zeros(n)
    lag = np.zeros(n)
    tr = np.zeros(n)
    
    for i in range(n):
        msd[i] = ((group.X.diff(periods=i)**2) + (group.X.diff(periods=i)**2)).mean()
        lag[i] = i*group.time_int.median()


    msdd = pd.DataFrame({'track_id':track_id[1], 'msd': msd, 'lag_time':lag})
    msdd['loc'] = track_id[2]
    msdd['filename']=track_id[0]
    
    msd_info = pd.concat([msd_info, msdd])
   


In [ ]:
#plot MSD curves
sns.lineplot(x=msd_info['lag_time'], y=msd_info['msd'], hue = msd_info['loc'], palette = 'magma')
plt.xlim(0,0.5)
plt.ylim(0,0.5)
plt.xlabel('T (sec)')
plt.ylabel('MSD (µm^2/s)')
plt.show()

In [ ]:
#----------anomalous diffusion analysis--------------

from scipy.optimize import curve_fit
D_alpha = pd.DataFrame()

def power_law(t, D, alpha):
    return 4* D * t**alpha


alpha_cal = msd_info.dropna()
for key,grp in alpha_cal.groupby(['filename','track_id', 'loc']):
    if len(grp)> 3:
        time = np.array(grp.lag_time)  # time data
        
        msd = np.array(grp.msd)  # MSD data
    
    
    
    popt, pcov = curve_fit(power_law, time, msd, maxfev=25000)
    D, alpha = popt

    values = pd.DataFrame({'filename': key[0], 'track_id': key[1], 'D': D, 'alpha':alpha, 'Loc':key[2] }, columns=['filename','track_id', 'D', 'alpha', 'Loc'], index=[0])
    D_alpha = pd.concat([D_alpha, values])
    
#plot alpha values
param = 'alpha'
sns.histplot(data = data, x = param, hue = 'Loc', bins=50, element="step", palette = 'magma',
             stat = 'density', common_norm=False, fill = True, kde = True, log_scale = False)


#statistics
mann_whit = stats.mannwhitneyu(data[data['Loc']=='inside'][param], data[data['Loc']=='outside'][param], nan_policy='omit')
plt.title('ER_mean: ' + str(data[data['Loc']=='outside'][param].mean()) + '\nLD_mean: ' + 
         str(data[data['Loc']=='inside'][param].mean()) + '\nER_median: ' + str(D_alpha[D_alpha['Loc']=='outside'][param].median()) + '\nLD_median: ' + 
         str(data[data['Loc']=='inside'][param].median()) + '\np: ' + str(mann_whit.pvalue))
plt.show()
    

In [ ]:
#--------apparent diffusion coefficient calculation-------------

def row_differences(group):
    # Compute differences within each group
    group['distance_sq'] = group['X'].diff(periods=period)**2 + group['Y'].diff(periods=period)**2
    group['time_int'] = group['time'].diff(periods=period)
    return group


MIN_track_info = MIN_track_info.sort_values(by=['track_id', 'time'])
MIN_track_info = MIN_track_info.groupby('track_id').apply(row_differences).drop(columns=['track_id']).reset_index()

MIN_track_info = MIN_track_info.groupby('track_id').filter(lambda x: x['time'].count() > 570).drop(columns=['level_1']) #longer than 50ms
MIN_track_info = MIN_track_info[(MIN_track_info['X']>=0) & (MIN_track_info['Y']>=0)] # remove any negative coordinates


in_D = (MIN_track_info[MIN_track_info['Loc'] == 'inside'].groupby(['filename', 'track_id']).distance_sq.mean().reset_index())
out_D = (MIN_track_info[MIN_track_info['Loc'] == 'outside'].groupby(['filename', 'track_id']).distance_sq.mean().reset_index())

in_D['time_int'] = (MIN_track_info[MIN_track_info['Loc'] == 'inside'].groupby(['filename', 'track_id']).time_int.median().reset_index().drop(columns=['filename', 'track_id']))
in_D['D'] = in_D.distance_sq/(in_D.time_int*4) #4Dt

out_D['time_int'] = (MIN_track_info[MIN_track_info['Loc'] == 'outside'].groupby(['filename', 'track_id']).time_int.median().reset_index().drop(columns=['filename', 'track_id']))
out_D['D'] = out_D.distance_sq/(out_D.time_int*4) #4Dt

in_D['Loc'] = 'inside'
out_D['Loc']='outside'

D_values = pd.concat([in_D, out_D])

#filter out the Dapp values bigger than 10µm^2/s
D_values = D_values[D_values.D<10]

#plot Dapp values
sns.histplot(data = D_values, x = 'D', hue = 'Loc', palette = 'magma_r', bins = 50, kde = False, element="step", stat = 'density', common_norm=False)

#statistical tests
mann_whit = stats.mannwhitneyu(in_D['D'], out_D['D'], nan_policy='omit')
t_test = stats.ttest_ind(in_D['D'], out_D['D'], nan_policy='omit')

plt.title('ER_D mean: ' + str(D_values[D_values['Loc']=='outside'].D.mean()) + '\nLD_D mean: ' + str(D_values[D_values['Loc']=='inside'].D.mean()) +
         '\nER_D median: ' + str(D_values[D_values['Loc']=='outside'].D.median()) + '\nLD_D median: ' + str(D_values[D_values['Loc']=='inside'].D.median()) + 
         '\np: ' + str(mann_whit.pvalue))

plt.xlim(0,8)
plt.show()

In [ ]:
#plot sampling rate

sns.histplot(MIN_track_info.dropna().time_int, bins = 500, color = 'grey')
plt.xlim(0,0.0008)
plt.title('median: ' + str(min_total.dropna().time_int.median()) + '\nmean: ' + str(min_total.dropna().time_int.mean()))
plt.show()